In [1]:
import random

GOAL = [
    [1, 2, 3],
    [6, 5, 4],
    [7, 8, 0],
]


# Nếu phép lai crosover tạo ra các số trùng lặp thì điều chỉnh lại kết quả để đảm báo đúng với yêu cầu (8 số là 1-8 và ô trống là số 0)
def AdjustState(state):
    state1d = Convert2DTo1DList(state)
    stateset = set(state1d)
    missingset = set([0, 1, 2, 3, 4, 5, 6, 7, 8])
    missingset.difference_update(stateset)
    remain = list(missingset)
    for i in range(len(state1d) - 1):
        for j in range(i + 1, len(state1d)):
            if state1d[i] == state1d[j]:
                state1d[j] = remain[0]
                remain.remove(remain[0])
                break
    return Convert1DTo2DList(state1d)


# Chuyển List 1 chiều sang list hai chiều
def Convert1DTo2DList(input_list):
    list2d = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    for i in range(3):
        for j in range(3):
            list2d[i][j] = input_list[i * 3 + j]
    return list2d


# Chuyển list 2 chiều sang list 1 chiều
def Convert2DTo1DList(input_list):
    list1d = []
    for i in range(3):
        for j in range(3):
            list1d.append(input_list[i][j])
    return list1d

# Tìm vị trí của ô trống (số 0)
def FindZero(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return [i, j]
    return None

# Hoán đổi vị trí 2 ô
def Swap(state, i, j, ii, jj):
    statecopy = state.copy()
    temp = statecopy[i][j]
    statecopy[i][j] = statecopy[ii][jj]
    statecopy[ii][jj] = temp
    return statecopy

# Chuyển list 2 chiều về dạng xâu để in ra đáp số bằng hàm print
def Convert2DListToString(input_list):
    return "\n".join(["".join(str(x)) for x in input_list]).replace("0", " ")


##################################
# Để sử dụng các thuật toán local search cần một hàm value để xác định giá trị của các thuật toán local search
# Với thuật toán Genetic Algorithm, cần yêu cầu thêm 3 hàm generate_random_state (tạo cấu hình ban đầu), crossover (lai tạo), mutate (tạo đột biến)
# Với thuật toán Hill Climbing, cần yêu cầu thêm 3 hàm actions, result và cost. Ngoài ra, còn cần initial_state để khởi tạo cấu hình ban đầu
class EightNumbersProblem:
    # Giá trị sẽ được xác định dựa trên số lượng các ô nằm đúng vị trí.
    # Càng nhiều ô nằm đúng vị trí thì càng tốt - đó là mục tiêu của các thuật toán local search
    def value(self, state):
        sum = 0
        for i in range(3):
            for j in range(3):
                if state[i][j] == GOAL[i][j]:
                    sum += 1
        return sum

    initial_state = [
        [1, 2, 3],
        [7, 6, 4],
        [5, 8, 0],
    ]

    # Hàm action cho các thuật toán local search khác GA
    def actions(self, state):
        actions = []
        [i, j] = FindZero(state)
        if i>0:
            actions.append([i-1, j])
        if i<2:
            actions.append([i+1, j])
        if j>0:
            actions.append([i, j-1])
        if j<2:
            actions.append([i, j+1])
        return actions

    #Thực thi result
    def result(self, state, action):
        new_state = state.copy()
        [i, j] = FindZero(state)
        Swap(new_state, action[0], action[1], i, j)
        return new_state

    #Tính cost cho mỗi lần dịch chuyển
    def cost(self, state, action, state2):
        return 1

    # Sinh cấu hình ban đầu một cách ngẫu nhiên
    # Tạo một list 1 chiều gồm 8 số và ô trống (số 0). Cho nhảy ngẫu nhiêu thứ tự bằng shuffle, sau đó chuyển list 1 chiều sang list 2 chiều
    def generate_random_state(self):
        orin = [1, 2, 3, 4, 5, 6, 7, 8, 0]
        random.shuffle(orin)
        init_state = Convert1DTo2DList(orin)
        return init_state

    # Lai tạo 2 cấu hình: cấu hình 1 chỉ lấy một dòng, cấu hình 2 lấy 2 dòng
    # Sau đó ghép lại để tạo ra list 2 chiều union
    # Tuy nhiên, trong union sẽ có các số trùng lặp, nên sử dụng AdjustState để chuyển đổi về dạng thức hợp lệ
    # Ví dụ, nếu union có dạng [[1 2 3], [3 5 6], [7 8 0]] (lưu ý số 3 lặp 2 lần và thiếu số 4)
    # Khi đó, số lặp lần thứ 2 sẽ lấy giá trị là số còn thiếu (nếu nhiều số lặp lại thì lấy theo thứ tự các số còn thiếu) => Xem chi tiết trong AdjustState
    def crossover(self, state1, state2):
        cut_first = state1[0][:]
        cut_second = state2[1:3][:]
        union = [cut_first, cut_second[0], cut_second[1]]
        union = AdjustState(union)
        return union

    # Phép tạo đột biến: sẽ sinh ngẫu nhiên một cặp chỉ số i, j và hoán đổi giá trị của 2 phần tử (i, j) và (j, i)
    # Để đảm bảo trong trường hợp nếu i=j vẫn có sự thay đổi cấu hình, thì khi i=j, ta sẽ tăng chỉ số i lên 1 (nếu vượt quá 2 thì sẽ quay về 0)
    def mutate(self, state):
        i, j = random.randint(0, 2), random.randint(0, 2)
        mutated = state.copy()
        if i == j:
            i = (i + 1) % 3
        temp = mutated[i][j]
        mutated[i][j] = mutated[j][i]
        mutated[j][i] = temp
        return mutated

    # Hàm biểu diễn trạng thái kết quả
    def state_representation(self, state):
        return Convert2DListToString(state)

In [2]:
from simpleai.search.local import genetic

problem = EightNumbersProblem()
result = genetic(problem, population_size=100, iterations_limit=50, mutation_chance=0.1)
print(result.state_representation())

[1, 2, 3]
[6, 5, 4]
[7, 8,  ]


In [55]:
from simpleai.search.local import hill_climbing

result2 = hill_climbing(problem)
print(result2.state_representation())

[1, 2, 3]
[7, 4, 5]
[8,  , 6]


In [73]:
from simpleai.search.local import simulated_annealing

result3 = simulated_annealing(problem)
print(result3.state_representation())

[1, 2, 3]
[7, 5, 6]
[4,  , 8]
